In [1]:
import os
# os.environ['XLA_FLAGS']='--xla_gpu_deterministic_reductions'
# os.environ['TF_CUDNN_DETERMINISTIC']='1'

import jax
import jax.numpy as jnp
from jax import random
import numpy as np
import tensorflow as tf
import logging
import matplotlib.pyplot as plt

In [2]:
from config import ConfigSchema
config = ConfigSchema(
    learning_rate = 0.001,
    momentum = 0.9,
    batch_size = 30,
    num_epochs = 10,
)

In [3]:
%load_ext autoreload
%autoreload 1

%aimport train_stead
%aimport model.model
%aimport data

models = model.model

In [4]:
weight_tie_pattern = tuple([0,*np.ones(7,dtype=int)])

perceiver = models.Perceiver(
    
    input_shape = (600,3),
    
    weight_tie_pattern = weight_tie_pattern,
                                 # depth of net. The shape of the final attention mechanism will be:
                                 #   depth * (cross attention -> self_per_cross_attn * self attention)
    
    num_freq_bands = 6,          # number of freq bands, with original value (2 * K + 1)
    max_freq = 20.,              # maximum frequency, hyperparameter depending on how fine the data is
    freq_base = 2,               

    num_latents = 64,            # number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim = 32,             # latent dimension
    
    cross_heads = 1,             # number of heads for cross attention. paper said 1
    cross_head_dim = 64,         # number of dimensions per cross attention head
    
    latent_heads = 8,            # number of heads for latent self attention, 8
    latent_head_dim = 16,        # number of dimensions per latent self attention head
    
    num_classes = 2,            # output number of classes
    attn_dropout = 0.,
    ff_dropout = 0.,
    fourier_encode_data = True,  # whether to auto-fourier encode the data, using the input_axis given. defaults to True, but can be turned off if you are fourier encoding the data yourself
    self_per_cross_attn = 2      # number of self attention blocks per cross attention
)

In [111]:
key = jax.random.PRNGKey(0)
key, subkey = jax.random.split(key)

trace = random.normal(subkey,(1, 6000, 3)) # 1 imagenet image, pixelized

out, params = perceiver.init_with_output(random.PRNGKey(0),trace)

out.shape


(1, 2)

In [133]:
output, mod_vars = perceiver.apply(params, batch_x, mutable='intermediates')
jax.nn.softmax(output).shape

(30, 2)

In [5]:
train_ds,test_ds = data.get_stead(batch_size=config.batch_size)

/home/jovyan/work/data.py:85: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  gen = stead_generator(hdf_file,csv_file,test_ratio,noise_ratio)


In [8]:
state = train_stead.train_and_evaluate(perceiver, train_ds, test_ds, config, workdir='../tensorboard')

0it [00:00, ?it/s]


Interrupted by user at epoch 2


In [162]:
from tqdm.notebook import tqdm

it = iter(test_ds)

epoch_loss = []
epoch_accuracy = []
for batch_x,batch_y in tqdm(it):
    _, loss, accuracy = train_stead.apply_model(perceiver, state, batch_x, batch_y)
    epoch_loss.append(loss)
    epoch_accuracy.append(accuracy)
    
val_loss = np.mean(epoch_loss)
val_accuracy = np.mean(epoch_accuracy)

  0%|          | 0/39 [00:00<?, ?it/s]